This scripts expects the following folder organization:
- home_folder
    - input_folder: contains: image files. Image files are tiff files with dimensions ch, x, y. Ch ('channel') has Nuclei (DAPI) at 4th position.

Outputs are saved in an output_folder inside home_directory

In [1]:
import numpy as np
import tifffile
import sklearn
from sklearn.preprocessing import minmax_scale, binarize
import matplotlib.pyplot as plt
import os
from PIL import Image
import imagej
import scyjava as sj
from scyjava import jimport
import xarray
import skimage
from skimage import io, exposure, feature
from skimage.io import imread, imshow
from skimage.measure import label, regionprops, regionprops_table
from skimage.filters import threshold_otsu
from scipy.ndimage import median_filter



In [2]:
#initialize pyImageJ with Fiji pulgins
sj.config.add_option('-Xmx6g') #use up to 15 extra gigabytes to process all the images in an experiment without running into memory overload error
ij = imagej.init('sc.fiji:fiji',add_legacy=True)
print(f"ImageJ2 version: {ij.getVersion()}") #NOTE: the version used for analysis was ImageJ2 version: 2.9.0/1.53t

#import ImagePlus object
ImagePlus = sj.jimport("ij.ImagePlus")


ImageJ2 version: 2.9.0/1.53t


In [3]:

#Indicate home_directory folder - home directory folder contains a sub-folder were images to analyse and PVN rois are contained
home_directory = r"/Users/alessandro_ulivi/ownCloud - alessandro.ulivi@lin-magdeburg.de@owncloud.gwdg.de/RNAscope/Analysis/Analysis_directory/Experiment4"

#Join home_directory folder with input_folder (Step_1_output) folder and output_folder (Step_2_output) - input_folder contains the images to analyse; the output_folder is where results are saved
path_input = os.path.join(home_directory, "Step_1_output")
path_output = os.path.join(home_directory, "Step_2_output")

#Create the output_folder (Step_2_output) folder, if not present
if not os.path.exists(path_output):
    os.makedirs(path_output)


#Generates a list of the objects in a given directory avoiding hidden files
def listdirNHF(path100):
    """
    forms a list of files in an directory, avoiding hidden files. Hidden files are identified by their names starting with a '.'
    inputs: directory folder
    outputs: list of all non-hidden elements in the input directory
    """
    #Initialize output list
    the_F_list = []
    
    #Collect input folder elements in a list
    start_list = os.listdir(path100)
    
    #Iterate through the elements of the list
    for item in start_list:
        
        #If the name of the element doesn't start with a '.', append it to the output list
        if not item.startswith('.'):
            the_F_list.append(item)
    
    #Return the output list
    return the_F_list



In [4]:
#define hyperparameters for the processing

#contrast stretching
min_percentil = 0.1 #Parameter used in the analysis: 0.1
max_percentil = 90 #Parameter used in the analysis: 90

#HyperSphereSmoothing - Edge-preserving smoothing
hypersphere_radius = 10 #Parameter used in the analysis is 10
filter_type = "filter.median" #Parameter used in the analysis is "filter.median"

# Watershed: mask to labeling.
ws_use_eight_connectivity = True #Parameter used in the analysis is True
ws_draw_watersheds = True #Parameter used in the analysis is True
ws_sigma = 8 #Parameter used in the analysis is 8

#mask erosion
erosion_dilation_times = 2 #Parameter used in the analysis is 2

#small roi removal (what is above is kept, below removed) - the unit is pixels
threshold_area_small_rois = 40 #Parameter used in the analysis is 40


In [5]:
#list files in input directory
list_files_input_dir = listdirNHF(path_input)

#Select .tif files in input directory
images_list = []
for f in list_files_input_dir:
    if str(f)[-4:]==".tif":
        images_list.append(f)


In [6]:
#define my_mode function. It will be used to rescale the intensity of nuclei images
def my_mode(arr_2D):
    """
    given a 2D numpy array, returns a single mode value into a list
    """
    vals_2Darr, counts_2Darr = np.unique(arr_2D, return_counts=True)
    mode_idx_2Darr = np.argwhere(counts_2Darr == np.max(counts_2Darr))
    mode_val_2Darr = vals_2Darr[mode_idx_2Darr].flatten().tolist()
    return mode_val_2Darr


#iterate the processing (among the images in the input folder)
for img in images_list:
    print("-------------------------------")
    print("working on image: ", str(img))
    
    #open image as numpy
    np_img = tifffile.imread(os.path.join(path_input, str(img)))
    
    # select nuclei channel
    np_dapi = np_img[3,:,:]
    
    #histograms rescaling
    min_np_dapi, max_np_dapi = np.percentile(np_dapi, (min_percentil, max_percentil)) #Calculate the max value of the intensity rescaling process
    mode_val_np_dapi = my_mode(np_dapi)[0] #Calculate the mode of value intensity histogram distribution
    equalized_np_dapi = exposure.rescale_intensity(np_dapi, in_range=(mode_val_np_dapi, max_np_dapi)) #Rescale intensity values on the mode-max range
    java_equalized_dapi = ij.py.to_java(equalized_np_dapi) #tranform rescaled image to java-compatible file
    
    # preprocess with edge-preserving smoothing
    HyperSphereShape = sj.jimport("net.imglib2.algorithm.neighborhood.HyperSphereShape")
    smoothed = ij.op().run("create.img", java_equalized_dapi)
    ij.op().run(filter_type, ij.py.jargs(smoothed, np_dapi, HyperSphereShape(hypersphere_radius)))
    # ij.py.show(smoothed)
    
    # otsu threshold to binary mask
    mask = ij.op().run("threshold.otsu", smoothed)
    # ij.py.show(mask)
    
    # fill holes of binary mask.
    fh_mask = ij.op().run("morphology.fillHoles", mask)
    # ij.py.show(fh_mask)
    
    # Watershed: mask to labeling
    args = ij.py.jargs(None, mask, ws_use_eight_connectivity, ws_draw_watersheds, ws_sigma, fh_mask)
    labeling = ij.op().run("image.watershed", args)
    # ij.py.show(labeling.getIndexImg(), cmap='tab10')
    
    #add second Watershed process
    mask_Plus = ij.py.to_imageplus(labeling)
    ij.IJ.run(mask_Plus, "8-bit", "")
    ij.IJ.run(mask_Plus, "Watershed", "")
    # ij.py.show(mask_Plus)
    
    #erode the mask before transforming to label-image
    for i in range(erosion_dilation_times):
        erodemask = ImagePlus("cells-mask", mask_Plus.createThresholdMask())
        ij.IJ.run(mask_Plus, "Erode", "")

    #transform to label image
    ij.IJ.run(erodemask, "Convert to Mask", "")
    xr_erodemask_mask = ij.py.from_java(erodemask)
    np_erodemask = np.asarray(xr_erodemask_mask)
    labeled_mask = label(np_erodemask)
    
    #filter small rois - first get image properties, second make a copy of the label image, third loop through the cell, and set the small ones to 0
    props = regionprops(labeled_mask) #get properties of label image objects
    labeled_mask_copy = labeled_mask #make a copy of label image
    for r in props: #Itearate through measured properties
        a = r['area'] #find area of the object
        if a<threshold_area_small_rois: #set values to 0 if area is smaller than threshold
            coordinates_a = r['coords']
            for c in coordinates_a:
                labeled_mask_copy[c[0],c[-1]]=0
    
    
    #binarize mask
    binary_np = binarize(labeled_mask_copy, threshold=0.0, copy=True)
    
    #convert to 0 and 255 and save as int16
    int_img_0_255 = np.where(binary_np>0, 255,0).astype(np.int16)
    
    #save result
    tifffile.imsave(os.path.join(path_output, str(img)),int_img_0_255)
    print("finished image: ", str(img))
    
    
print("finished")


-------------------------------
working on image:  F_16_a_fron_MAX_cut.tif


Operating in headless mode - the original ImageJ will have limited functionality.
Operating in headless mode - the IJ class will not be fully functional.


finished image:  F_16_a_fron_MAX_cut.tif
-------------------------------
working on image:  F_16_b_fron_MAX_cut.tif
finished image:  F_16_b_fron_MAX_cut.tif
finished
